In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
import matplotlib as mpl
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot 
from datetime import datetime, timedelta
import warnings
import datetime
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)
mpl.style.use('seaborn')
pd.set_option('display.max_columns', 1000)
from sklearn.metrics import accuracy_score


In [47]:
df=pd.read_csv("aud_usd_h4_h1_all_merged_corr_0.1_v1.csv")
df.columns = df.columns.str.replace('.','_')

In [48]:
df.head()

Unnamed: 0                 time_local  target1_x  target2_x  target3_x  \
0           0  2010-01-05 18:00:00+00:00     0.9117     0.9135     0.9142   
1           1  2010-01-05 22:00:00+00:00     0.9135     0.9142     0.9142   
2           2  2010-01-06 02:00:00+00:00     0.9142     0.9142     0.9156   
3           3  2010-01-06 06:00:00+00:00     0.9142     0.9156     0.9184   
4           4  2010-01-06 10:00:00+00:00     0.9156     0.9184     0.9196   

   bid_close_x  Volatility_USD_hourly  PC_macro_economic_ind_USD_hourly  \
0       0.9124                    0.0                          0.000000   
1       0.9117                    0.5                         -1.543434   
2       0.9135                    0.5                         -1.543434   
3       0.9142                    0.5                         -1.543434   
4       0.9142                    0.5                         -1.543434   

   Volatility_USD_daily  PC_macro_economic_ind_USD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   Volatility_AUD_hourly  PC_macro_economic_ind_AUD_hourly  \
0                    0.0                          0.000000   
1                    0.5                         -1.543434   
2                    0.5                         -1.543434   
3                    0.5                         -1.543434   
4                    0.5                         -1.543434   

   Volatility_AUD_daily  PC_macro_economic_ind_AUD_daily  \
0                   1.0                         0.051546   
1                   1.0                         0.051546   
2                   1.0                        -1.050000   
3                   1.0                        -1.050000   
4                   1.0                        -1.050000   

   ANZ Job Advertisements_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   AiG Performance of Mfg Index_PC_AUD_hourly  \
0                                         0.0   
1                                         0.0   
2                                         0.0   
3                                         0.0   
4                                         0.0   

   Building Permits (MoM)_PC_AUD_hourly  Building Permits (YoY)_PC_AUD_hourly  \
0                              0.000000                              0.000000   
1                             -4.484536                              1.397668   
2                             -4.484536                              1.397668   
3                             -4.484536                              1.397668   
4                             -4.484536                              1.397668   

   CFTC AUD NC net positions_PC_AUD_hourly  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   Construction Work Done_PC_AUD_hourly  \
0                                   0.0   
1                                   0.0   
2                                   0.0   
3                                   0.0   
4                                   0.0   

   Consumer Price Index (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       0.0   
2                                       0.0   
3                                       0.0   
4                                       0.0   

   Dwelling Unit Starts (QoQ)_PC_AUD_hourly  \
0                                       0.0   
1                                       

In [49]:
df=df.dropna(how='all',axis=1)
df.shape

(14028, 839)

In [50]:
df['time_local']=df['time_local'].astype(str).str[:-6]

In [51]:
df['year']=pd.DatetimeIndex(df['time_local']).year
df['month']=pd.DatetimeIndex(df['time_local']).month
df['day']=pd.DatetimeIndex(df['time_local']).day
df['hour']=pd.DatetimeIndex(df['time_local']).hour
df['dayofweek']=pd.DatetimeIndex(df['time_local']).dayofweek



In [52]:
import datetime   
#date = datetime.date.today()
date=pd.datetime.now().strftime("%Y-%m-%d %I:%M:%S")
date

'2019-04-25 01:00:24'

In [53]:
df['today']=date
df['today'] =pd.DatetimeIndex(df['today'])

In [54]:
df['nb_months'] = ((df['today'] - pd.to_datetime(df['time_local'] ))/np.timedelta64(1, 'M'))
df['nb_months'] = df['nb_months'].astype(int)

In [55]:
df['class_target_7'] = ((df['target1_x']-df['close_h1_7'])/df['close_h1_7'])*100
df.loc[df['class_target_7'] < 0.0005, 'new_class_target'] = -1
df.loc[df['class_target_7'] > 0.0005, 'new_class_target'] = 1
df.loc[df['class_target_7'] == 0, 'new_class_target'] = 0
#class_target3

In [56]:
col_drop= df.filter(regex='time').columns

In [57]:
target_cols = df.filter(regex='target').columns

In [58]:
cols_x = df.filter(regex='_x').columns

In [59]:
cols_y = df.filter(regex='_y').columns

In [72]:
X=df.drop(['Unnamed: 0','time_local', 'target2_x', 'target3_x', 'target1_y', 'target2_y',
       'target3_y', 'class_target2', 'class_target3',
       'PC_AUD_H4_target1', 'bid_close_y', 'today','target1_x'],axis=1)

In [73]:
X=X.fillna(0)

In [74]:
df_temp=X.select_dtypes(include=['object'])

In [75]:
obj=df_temp.loc[:, df_temp.dtypes =='object']
for col in obj:
    obj[col] = pd.to_numeric(obj[col], errors='coerce')
df_temp[obj.columns]=obj

In [76]:
df_temp=df_temp.replace([np.inf, -np.inf], np.nan)
df_temp=df_temp.fillna(0)
X[df_temp.columns]=df_temp

In [77]:
#X_final= X[X['year'] < 2018]
s=X[['year','month', 'dayofweek']]
for each in s:
    dummies = pd.get_dummies(s[each], prefix=each, drop_first=False)
    X=X.drop([each],axis=1)
    X = pd.concat([X, dummies], axis=1)

In [78]:
X=X[X.replace([np.inf, -np.inf], np.nan).notnull().all(axis=1)]

In [79]:
X_pred=X[(X['year_2019']==1) | (X['year_2018']==1)]
X_pred.to_csv("X_close_7_2018_25_4_2019_three_class.csv")

In [80]:
X_final=X[(X['year_2019']!=1) & (X['year_2018']!=1) ]

In [81]:
y=X_final['new_class_target']
X1=X_final.drop(['class_target1','new_class_target','class_target_7'],axis=1)

In [24]:
#from sklearn.preprocessing import StandardScaler
#scaler=StandardScaler()
#scaler.fit(X1)
#X1=scaler.transform(X1)

In [82]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
np.seed=0
from sklearn.linear_model import LogisticRegression

In [83]:
from imblearn.over_sampling import SMOTE
smt = SMOTE(random_state=1)
X2, y1 = smt.fit_sample(X1, y)
len(X2),len(y1)

(17208, 17208)

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X2, y1, test_size=0.3,random_state=1,stratify=y1)
print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)


(12045, 857) (12045,)
(5163, 857) (5163,)


In [85]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier()
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [86]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.48      0.50      0.49      1721
         0.0       0.96      0.91      0.93      1721
         1.0       0.47      0.48      0.48      1721

   micro avg       0.63      0.63      0.63      5163
   macro avg       0.64      0.63      0.63      5163
weighted avg       0.64      0.63      0.63      5163



In [87]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.70      0.74      0.72      4015
         0.0       0.96      0.90      0.93      4015
         1.0       0.71      0.71      0.71      4015

   micro avg       0.78      0.78      0.78     12045
   macro avg       0.79      0.78      0.78     12045
weighted avg       0.79      0.78      0.78     12045



In [88]:
accuracy_score(y_test,y_pred)

0.6300600426108851

In [89]:
accuracy_score(y_train,gb.predict(X_train))

0.7823993358239933

In [33]:
pd.crosstab(y_test,y_pred)

col_0  -1.0   0.0   1.0
row_0                  
-1.0    814    41   866
 0.0     72  1563    86
 1.0    830    55   836

In [90]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.8,n_estimators=80)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)

In [91]:
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.557040480340887, 0.5793275217932752)

In [48]:

from sklearn.model_selection import GridSearchCV
parameters = {
    "learning_rate": [0.1],
    "max_depth":[3,5,8],
    "criterion": ["friedman_mse"],
    "subsample":[0.8,0.9,1.0],
    "n_estimators":[90,100,120]
    }

clf = GridSearchCV(gb, parameters, cv=5,verbose=5)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.6331950207468879, total= 5.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.9min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.6355334163553341, total= 6.6min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 18.7min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.6081361560813615, total= 6.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 25.7min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.8, score=0.632890365448505, total= 5.5min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6286307053941909, total= 6.3min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6255707762557078, total= 7.0min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9, score=0.6334578663345787, total= 5.8min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=90, subsample=0.9,

KeyboardInterrupt: 

In [49]:

from sklearn.model_selection import GridSearchCV
parameters = {
    "learning_rate": [0.1],
    "max_depth":[3,5,8],
    "criterion": ["friedman_mse"],
    "subsample":[0.8,0.9],
    "n_estimators":[100,120]
    }

clf = GridSearchCV(gb, parameters, cv=5,verbose=3)
clf.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6261410788381743, total= 7.8min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  7.8min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6247405562474055, total= 7.6min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 15.3min remaining:    0.0s


[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6330427563304276, total= 7.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6164383561643836, total= 8.1min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8, score=0.6399501661129569, total= 7.9min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9, score=0.637759336099585, total= 8.5min
[CV] criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.9 
[CV]  criterion=friedman_mse, learning_rate=0.1, max_depth=3, n_estimators=100, subsa

KeyboardInterrupt: 

In [52]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.8,n_estimators=120)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6380011621150494, 0.8048152760481527)

In [53]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.8,n_estimators=90)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6261863257795856, 0.7813200498132005)

In [56]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.1,subsample=0.6,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.6246368390470657, 0.7901203819012038)

In [58]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=100)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5541351927174124, 0.593607305936073)

In [59]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=100,max_depth=4)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5828006972690296, 0.6596928185969282)

# Final model

In [92]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(learning_rate=0.01,subsample=0.6,n_estimators=120,max_depth=3)
gb.fit(X_train,y_train)
y_pred=gb.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,gb.predict(X_train))

(0.5748595777648654, 0.6152760481527605)

In [93]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.44      0.61      0.51      1721
         0.0       0.94      0.75      0.83      1721
         1.0       0.45      0.37      0.41      1721

   micro avg       0.57      0.57      0.57      5163
   macro avg       0.61      0.57      0.58      5163
weighted avg       0.61      0.57      0.58      5163



In [94]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.48      0.67      0.56      4015
         0.0       0.94      0.74      0.82      4015
         1.0       0.54      0.44      0.48      4015

   micro avg       0.62      0.62      0.62     12045
   macro avg       0.65      0.62      0.62     12045
weighted avg       0.65      0.62      0.62     12045



In [95]:
pd.crosstab(y_test,y_pred)

col_0  -1.0   0.0   1.0
row_0                  
-1.0   1042    33   646
 0.0    307  1287   127
 1.0   1034    48   639

In [96]:
pd.crosstab(y_train,gb.predict(X_train))

col_0  -1.0   0.0   1.0
row_0                  
-1.0   2686    81  1248
 0.0    770  2956   289
 1.0   2131   115  1769

In [102]:
from joblib import dump,load
dump(gb,'gb_close_7_3_class.joblib') 

['gb_close_7_3_class.joblib']

In [ ]:
######### Testing other models

In [97]:
from xgboost import XGBClassifier
model = XGBClassifier(eta=0.01,subsample=0.6,colsample_bytree=0.6,reg_lambda=2)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.63141584350184, 0.7596513075965131)

In [98]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

        -1.0       0.48      0.49      0.49      1721
         0.0       0.96      0.90      0.93      1721
         1.0       0.48      0.50      0.49      1721

   micro avg       0.63      0.63      0.63      5163
   macro avg       0.64      0.63      0.64      5163
weighted avg       0.64      0.63      0.64      5163



In [99]:
print(classification_report(y_train,gb.predict(X_train)))

              precision    recall  f1-score   support

        -1.0       0.48      0.67      0.56      4015
         0.0       0.94      0.74      0.82      4015
         1.0       0.54      0.44      0.48      4015

   micro avg       0.62      0.62      0.62     12045
   macro avg       0.65      0.62      0.62     12045
weighted avg       0.65      0.62      0.62     12045



In [100]:
from xgboost import XGBClassifier
model = XGBClassifier(eta=0.01,subsample=0.8,colsample_bytree=0.8)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.6232810381561108, 0.7655458696554587)

In [101]:
from xgboost import XGBClassifier
model = XGBClassifier(eta=0.01,subsample=0.8,colsample_bytree=0.8,reg_lambda=0.8)
model.fit(X_train, y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred),accuracy_score(y_train,model.predict(X_train))

(0.6287042417199302, 0.7692818596928186)